In [27]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="AzureProject")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-241963
Azure region: westeurope
Subscription id: 61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30
Resource group: aml-quickstarts-241963


In [28]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "AzureProject"

# TODO: Create compute cluster
# Use vm_size = " Standard_D2_V2"in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# verify that cluser does not exist already
try:
    cpu_cluster=ComputeTarget(workspace=ws, name= cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster= ComputeTarget.create(ws,cpu_cluster_name,compute_config)

cpu_cluster.wait_for_completion(show_output=True)
# print detailed status for the current cluster
print(cpu_cluster.get_status().serialize())
    

Found existing cluster, use it.

Running
{'errors': [], 'creationTime': '2023-09-28T13:43:21.534368+00:00', 'createdBy': {'userObjectId': '92a5563a-8782-4728-a447-f9eee0fbc083', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': None}, 'modifiedTime': '2023-09-28T13:48:25.942271+00:00', 'state': 'Running', 'vmSize': 'STANDARD_D2S_V3'}


In [31]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{'--C':choice(0.01,0.1,1,10,50,100,500,1000),
'--max_iter':choice(50,100,200,300)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1,slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                     script='train.py',
                     compute_target=cpu_cluster,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=16,
                                    max_concurrent_runs=3)

In [32]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run= exp.submit(hyperdrive_config,show_output=True)
RunDetails(hyperdrive_run).show()

hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_02f8fdc5-4396-402c-8718-782d45f806fa
Web View: https://ml.azure.com/runs/HD_02f8fdc5-4396-402c-8718-782d45f806fa?wsid=/subscriptions/61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30/resourcegroups/aml-quickstarts-241963/workspaces/quick-starts-ws-241963&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-09-28T15:16:44.639764][GENERATOR][INFO]Trying to sample '3' jobs from the hyperparameter space
[2023-09-28T15:16:45.494050][GENERATOR][INFO]Successfully sampled '3' jobs, they will soon be submitted to the execution target.
[2023-09-28T15:16:45.6053628Z][SCHEDULER][INFO]Scheduling job, id='HD_02f8fdc5-4396-402c-8718-782d45f806fa_0' 
[2023-09-28T15:16:45.8938652Z][SCHEDULER][INFO]Scheduling job, id='HD_02f8fdc5-4396-402c-8718-782d45f806fa_2' 
[2023-09-28T15:16:45.8949504Z][SCHEDULER][INFO]Scheduling job, id='HD_02f8fdc5-4396-402c-8718-782d45f806fa_1' 
[2023-09-28T15:16:46.4627840Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_02f8fdc5-4396-402c-8

{'runId': 'HD_02f8fdc5-4396-402c-8718-782d45f806fa',
 'target': 'AzureProject',
 'status': 'Completed',
 'startTimeUtc': '2023-09-28T15:16:43.277547Z',
 'endTimeUtc': '2023-09-28T15:25:48.443262Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '47e1db4f-2e0f-4e0f-bc01-ff1b8186acb2',
  'user_agent': 'python/3.8.5 (Linux-5.15.0-1040-azure-x86_64-with-glibc2.10) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.51.0',
  'space_size': '32',
  'score': '0.9176024279210926',
  'best_child_run_id': 'HD_02f8fdc5-4396-402c-8718-782d45f806fa_2',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_02f8fdc5-4396-402c-8718-782d45f806fa_2'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClient

In [33]:
import joblib
# Get your best run and save the model from that run.
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0,reverse =False, discard_no_metric=False))
### YOUR CODE HERE ###
best_run=hyperdrive_run.get_best_run_by_primary_metric()
print("Best run metric:", best_run.get_metrics())
print("Run details",best_run.get_details() )
print("Best run file names", best_run.get_file_names())

[{'run_id': 'HD_02f8fdc5-4396-402c-8718-782d45f806fa_2', 'hyperparameters': '{"--C": 10, "--max_iter": 300}', 'best_primary_metric': 0.9176024279210926, 'status': 'Completed'}, {'run_id': 'HD_02f8fdc5-4396-402c-8718-782d45f806fa_13', 'hyperparameters': '{"--C": 10, "--max_iter": 200}', 'best_primary_metric': 0.917298937784522, 'status': 'Completed'}, {'run_id': 'HD_02f8fdc5-4396-402c-8718-782d45f806fa_9', 'hyperparameters': '{"--C": 1, "--max_iter": 300}', 'best_primary_metric': 0.917298937784522, 'status': 'Completed'}, {'run_id': 'HD_02f8fdc5-4396-402c-8718-782d45f806fa_15', 'hyperparameters': '{"--C": 0.1, "--max_iter": 200}', 'best_primary_metric': 0.9171471927162367, 'status': 'Completed'}, {'run_id': 'HD_02f8fdc5-4396-402c-8718-782d45f806fa_6', 'hyperparameters': '{"--C": 10, "--max_iter": 100}', 'best_primary_metric': 0.9171471927162367, 'status': 'Completed'}, {'run_id': 'HD_02f8fdc5-4396-402c-8718-782d45f806fa_12', 'hyperparameters': '{"--C": 500, "--max_iter": 300}', 'best_pr

In [37]:
best_run.get_file_names()

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2023-09-28-15',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/lifecycler/vm-bootstrapper.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [36]:
hypr_mdl='hyperdrive_model'
folder='output1'
path=os.path.join(folder,'model.joblib')
hyperdrive_model=best_run.register_model(model_name=hypr_mdl, model_path=path)

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path output1/model.joblib in the set of files uploaded to the run: ['logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2023-09-28-15', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/lifecycler/vm-bootstrapper.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path output1/model.joblib in the set of files uploaded to the run: ['logs/azureml/dataprep/0/backgroundProcess.l

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
ds=TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])

In [ ]:
import os

os.getcwd()

In [ ]:
os.listdir()

In [8]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='Accuracy',
    training_data=x,
    label_column_name=y,
    n_cross_validations=5)

In [12]:
# Submit your automl run
run=exp.submit(config=automl_config, show_output=True)


ConfigException: ConfigException:
	Message: Argument [label_column_name] is of unsupported type: [<class 'pandas.core.series.Series'>]. Supported type(s): [int, str]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Argument [label_column_name] is of unsupported type: [<class 'pandas.core.series.Series'>]. Supported type(s): [int, str]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "label_column_name",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid"
            }
        },
        "reference_code": "061ed905-e59e-42b9-ad95-cd18f40b4358"
    }
}

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model=run.get_output()
print(best_run)
print(fitted_model)

print("Best run metrics :", best_run.get_metrics())
print("Best run details :", best_run.get_details())